In [65]:
#brainfuck parser v2

braincleanse = """
#takes input as comma separated list:;
#e.g.: 9,4                           ;
offset = 48;
input(a);
input(idc);
input(b);
a-=offset;
b-=offset;
c = a/b;
c += offset;
@rem += offset;
remainderWeCareAbout = @rem;
print(c);
delete(c);
#now just print " r";
space = offset;
offsetThird = offset/3;
space -=offsetThird;
print(space);
r = 2 * offset;
halfSpace = space / 2;
r += halfSpace;
r += 2;
print(r);
print(remainderWeCareAbout);
"""

dataPointer = 0
#all other are guaranteed to be 0
#@ is for temp variables
#@rem is the remainder after division operation
#print prints the variable inside
#delete deletes the variable inside
variables = {}
brainfuckCommands = []

#cannot use @ and |
validOperators = ["=", "+", "-", "*", "/", "%", "^", ",", ";", ":", "[", "]", "{", "}", "(", ")"]

def codeError(msg):
    print("ERROR: " + msg)

def isnumber(x):
    try:
        int(x)
        return True
    except ValueError:
        return False
    
#code checks before compilation to brainfuck
def staticCheck(i):
    original = i + ";"
    string = i
    for i in validOperators:
        string = string.replace(i, "|")
    l = string.split("|")
    r = []
    for i in l:
        if not isnumber(i) and i is not "":
            r.append(i)  
    if len(r) != len(set(r)):
        #contains duplicates
        codeError(original + "\ncontains duplicate variables. Each variable can be used only one in one line")
        1/0

braincleanse = braincleanse.replace('\n',"")
dirtyLines = braincleanse.split(";")
lines = []
for i in dirtyLines:
    if (i != "" and ("=" in i or "(" in i or "{" in i or "}" in i) and not "#" in i):
        i = i.replace(" ", "")
        staticCheck(i)
        lines.append(i)
        
def allocateVariableAt(var, preferredIndex, size):
    #chance to fill previous
    cell = preferredIndex - size
    while True:
        if cell < 0:
            cell = 0
            continue
        free = True
        for x in range(size):
            for i in variables:
                if variables[i]["index"] == cell + x:
                    free = False
                    break
        if free:
            realloc = var in variables
            variables[var] = {"index" : cell, "size": size}
            print("reallocating" if realloc else "allocating new" + " variable:", var, "at location:", cell)
            break
        cell += 1
        
#around dataPointer
def allocateVariable(var, size):
    allocateVariableAt(var, dataPointer, size)

def move(dst, src):
    fetchData(getvar(src)["index"])
    brainfuckCommands.append("[")
    fetchData(getvar(dst)["index"])
    brainfuckCommands.append("+")
    fetchData(getvar(src)["index"])
    brainfuckCommands.append("-]")
    
#doesnt look for free index, forcefully uses the specified index
def forceAllocateVariable(var, index, size):
    variables[var] = {"index" : index, "size": size}

def getvar(key):
    arr = key.find("[")
    if arr == -1:
        return variables[key]
    
    end = key.find("]")
    expression = key[arr + 1 : end]
    
    if isnumber(expression):
        k = key[:arr]
        return {"index": variables[k]["index"] + int(expression), "size": 1}
    else:
        #TODO dynamic data pointer
        pass

#if dst is array accessed, has to copy
def setvar(dst, src):
    arr = dst.find("[")
    if arr == -1:
        variables[dst] = getvar(src)
    else:
        move(dst, src)   
    
#for temps
def freeVariable(var):
    del variables[var]
    print("freed variable:", var)

def parseLine(string):
    found = string.find("=")
    specialOperators = ['+', '-']
    if found != -1:
        if string[found - 1] in specialOperators:
            return (string[0:found - 1], string[found - 1:])
        else:
            return (string[0:found], string[found+1:])
    else:
        return("", string)
    
def fetchData(index):
    global dataPointer
    assert(index >= 0)
    move = index - dataPointer
    dataPointer = index
    if move < 0:
        brainfuckCommands.append("<" * (move * -1))
    else:
        brainfuckCommands.append(">" * move)
        
def changeData(n):
    if n < 0:
        brainfuckCommands.append("-" * (n * -1))
    else:
        brainfuckCommands.append("+" * n)
    
###BEGIN OPERATORS###
def increment(context):
    if context[1].startswith("+="):
        context = (context[0],context[1][2:], context[2])
        #if not numeric, do this, else do '+' increment
        if isnumber(context[1]):
            assignConstant(context)
        else:
            allocateVariable("@", 1)
            fetchData(getvar(context[1])["index"])
            brainfuckCommands.append("[")
            fetchData(getvar(context[0])["index"])
            brainfuckCommands.append("+")
            fetchData(getvar("@")["index"])
            brainfuckCommands.append("+")
            fetchData(getvar(context[1])["index"])
            brainfuckCommands.append("-]")
            
            setvar(context[1], "@")
            freeVariable("@") 
        return True
    return False
            
def decrement(context):
    if context[1].startswith("-="):
        context = (context[0],context[1][2:], context[2])
        #if not numeric, do this, else do '-' decrement
        if isnumber(context[1]):
            if not context[1].startswith('-'):
                context = (context[0],"-" + context[1])
            else:
                context = (context[0],context[1][1:])
            assignConstant(context)
        else:
            xIndex = getvar(context[1])["index"]
            allocateVariable(context[1], 1)
            fetchData(xIndex)
            brainfuckCommands.append("[")
            fetchData(getvar(context[0])["index"])
            brainfuckCommands.append("-")
            fetchData(getvar(context[1])["index"])
            brainfuckCommands.append("+")
            fetchData(xIndex)
            brainfuckCommands.append("-]")
        return True
    return False

def add(context):
    plusIndex = context[1].find("+")
    if plusIndex != -1:
        x0 = context[1][:plusIndex]
        x1 = context[1][plusIndex + 1:]
        if x0 == "" or x1 == "":
            return False
        #clear if previously held value
        if context[2]:
            fetchData(getvar(context[0])["index"])
            brainfuckCommands.append("[-]")
        increment((context[0], "+=" + x0, context[2]))
        increment((context[0], "+=" + x1, context[2]))
        return True
    return False
        
def substract(context):
    minusIndex = context[1].find("-")
    if minusIndex != -1:
        x0 = context[1][:minusIndex]
        x1 = context[1][minusIndex + 1:]
        if x0 == "" or x1 == "":
            return False
        #clear if previously held value
        if context[2]:
            fetchData(getvar(context[0])["index"])
            brainfuckCommands.append("[-]")
        increment((context[0], "+=" + x0, context[2]))
        decrement((context[0], "-=" + x1, context[2]))
        return True
    return False

def getInput(context):
    if context[1].find("input(") != -1:
        expression = context[1][6:-1]
        allocateVariable(expression, 1)
        fetchData(getvar(expression)["index"])
        brainfuckCommands.append(',')
        return True
    return False

def printOutput(context):
    if context[1].find("print(") != -1:
        expression = context[1][6:-1]
        if isnumber(expression):
            allocateVariable("@", 1)
            fetchData(getvar("@")["index"])
            changeData(int(expression))
            freeVariable("@")
        else:
            fetchData(variables[expression]["index"])
        brainfuckCommands.append('.')
        return True
    return False

def multiply(context):
    multiplyIndex = context[1].find("*")
    if multiplyIndex != -1:
        x0 = context[1][:multiplyIndex]
        x1 = context[1][multiplyIndex + 1 :]
        x0type = isnumber(x0) #true if is number
        x1type = isnumber(x1)
        
        def numMultNum(num0, num1):
            if num1 < num0:
                num0, num1 = num1, num0
            #num0 is always smaller here
            allocateVariable("@", 1)
            fetchData(getvar("@")["index"])
            changeData(num0)
            brainfuckCommands.append("[")
            fetchData(getvar(context[0])["index"])
            changeData(num1)
            fetchData(getvar("@")["index"])
            brainfuckCommands.append("-]")
            freeVariable("@")
            
        def numMultVar(num, var):
            previousVarPosition = getvar(var)["index"]
            allocateVariable(var, 1)
            fetchData(previousVarPosition)
            brainfuckCommands.append("[")
            fetchData(getvar(var)["index"])
            brainfuckCommands.append("+")
            fetchData(getvar(context[0])["index"])
            changeData(num)
            fetchData(previousVarPosition)
            brainfuckCommands.append("-]")
            
        def varMultVar(var0, var1):
            #really important its as close as possible
            allocateVariableAt("@", getvar(var1)["index"], 1)
            allocateVariableAt("@@", getvar(var0)["index"], 1)
            
            #do this var0 times
            fetchData(getvar(var0)["index"])
            brainfuckCommands.append("[")
            fetchData(getvar("@@")["index"])
            brainfuckCommands.append("+")
            #outer loop start
            #inner loop, add var1 to result and move var1
            #add var1 to result
            fetchData(getvar(var1)["index"])
            brainfuckCommands.append("[")
            fetchData(getvar("@")["index"])
            brainfuckCommands.append("+")
            fetchData(getvar(context[0]["index"]))
            brainfuckCommands.append("+")
            fetchData(getvar(var1)["index"])
            brainfuckCommands.append("-]")
            #move value from @ to var1
            fetchData(getvar("@")["index"])
            brainfuckCommands.append("[")
            fetchData(getvar(var1)["index"])
            brainfuckCommands.append("+")
            fetchData(getvar("@")["index"])
            brainfuckCommands.append("-]")
            #outer loop end
            fetchData(getvar(var0)["index"])
            brainfuckCommands.append("-]")
            
            #var0 moved to @@
            # @@ already cleanup here
            setvar(var0, "@@")
            freeVariable("@@") 
            #cleanup
            freeVariable("@")
        
        if x0type and x1type:
            numMultNum(int(x0), int(x1))
        elif not x0type and x1type:
            numMultVar(int(x1), x0)
        elif x0type and not x1type:
            numMultVar(int(x0), x1)
        else:
            varMultVar(x0, x1)
        
        return True
    return False

def delete(context):
    if context[1].find("delete(") != -1:
        expression = context[1][7:-1]
        if expression in variables:
            #clear it to 0!
            var = getvar(expression)
            for i in range(var["size"]):
                fetchData(var["index"] + i)
                brainfuckCommands.append("[-]")
            freeVariable(expression)
        return True
    return False

def assignConstant(context):
    if isnumber(context[1]):
        fetchData(getvar(context[0])["index"])
        changeData(int(context[1]))
    else:
        increment((context[0], "+=" + context[1], context[2]))
    return True

def divide(context):
    divideIndex = context[1].find("/")
    if divideIndex != -1:
        x0 = context[1][:divideIndex]
        x1 = context[1][divideIndex + 1 :]
        
        #firstly allocate space needed
        allocateVariable("@native_division", 6)
        
        #n/d, n%d
        #move n into place
        fetchData(getvar(x0)["index"])
        brainfuckCommands.append("[")
        fetchData(getvar("@native_division[0]")["index"])
        brainfuckCommands.append("+")
        fetchData(getvar(x0)["index"])
        brainfuckCommands.append("-]")
        #new x0 pos
        setvar(x0, "@native_division[1]")
        
        #move d into place, needs copy so increment
        increment(("@native_division[2]", "+=" + x1, False))
        
        #move data pointer into place
        fetchData(getvar("@native_division[0]")["index"])
        
        #the divmod algorithm itself
        brainfuckCommands.append("[->+>-[>+>>]>[+[-<+>]>+>>]<<<<<<]")
        
        #returns division
        forceAllocateVariable(context[0], getvar("@native_division[4]")["index"], 1)
        #puts remainder in @rem, keeps it till overwritten by another division
        forceAllocateVariable("@rem", getvar("@native_division[3]")["index"], 1)
        
        #clear the remaining unused field
        fetchData(getvar("@native_division[2]")["index"])
        brainfuckCommands.append("[-]")
        
        #free it
        freeVariable("@native_division")
        
        return True
    return False
###END OPERATORS###
    
for i in lines:
    parsed = parseLine(i)
    if parsed[0] != "" and parsed[0] not in variables:
        allocateVariable(parsed[0], 1)
        parsed = (parsed[0], parsed[1], False)
    else:
        parsed = (parsed[0], parsed[1], True)
    
    #check for possible operations
    #[]= funcs
    if increment(parsed):
        continue
    if decrement(parsed):
        continue
    #certainly clear result if needed, dont do it for increment, decrement and other []= funcs
    if parsed[0] != "" and parsed[2] == True:
        fetchData(variables[parsed[0]]["index"])
        brainfuckCommands.append("[-]")
        
    if add(parsed):
        continue
    if getInput(parsed):
        continue
    if printOutput(parsed):
        continue
    if multiply(parsed):
        continue
    if divide(parsed):
        continue
    if delete(parsed):
        continue
    if substract(parsed):
        continue
    if assignConstant(parsed):#always last!
        continue
    
brainfuck = "".join(brainfuckCommands)
print("\n\n######################OUTPUT#########################")
print(brainfuck)
print(variables)

allocating new variable: offset at location: 0
allocating new variable: a at location: 1
allocating new variable: idc at location: 2
allocating new variable: b at location: 3
reallocating offset at location: 4
reallocating offset at location: 0
allocating new variable: c at location: 4
allocating new variable: @native_division at location: 5
allocating new variable: @ at location: 1
freed variable: @
freed variable: @native_division
allocating new variable: @ at location: 7
freed variable: @
allocating new variable: @ at location: 0
freed variable: @
allocating new variable: remainderWeCareAbout at location: 7
allocating new variable: @ at location: 10
freed variable: @
freed variable: c
allocating new variable: space at location: 8
allocating new variable: @ at location: 9
freed variable: @
allocating new variable: offsetThird at location: 0
allocating new variable: @native_division at location: 11
freed variable: @native_division
reallocating offsetThird at location: 13
allocating ne